In [ ]:
### This is a GPU implementation for the HQC classifier using Scikit-learn's methods, but with PyTorch as the backend. ###

In [ ]:
# I have implemented the code below in such a way that you would only need to input X and y as numpy arrays and the
# output y_hat would also be a numpy array (rather than PyTorch tensors). This would make it easier to use the package
# below with minimal knowledge of PyTorch tensors.

# Take note of the parameter n_splits, where the implementation of n_splits now is different to the one in the CPU case.
# Please read the description of n_splits below.

In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.multiclass import check_classification_targets
import torch
from torch.nn.functional import normalize
from scipy import linalg

class PGMHQC_gpu_dtype(BaseEstimator, ClassifierMixin):
    """The Pretty Good Measurement (PGM) - Helstrom Quantum Centroid (HQC) classifier is a 
    quantum-inspired supervised classification approach for data with multiple classes.
                         
    Parameters
    ----------
    rescale : int or float, default = 1
        The dataset rescaling factor. A parameter used for rescaling the dataset. 
    encoding : str, default = 'amplit'
        The encoding method used to encode vectors into quantum densities. Possible values:
        'amplit', 'stereo'. 'amplit' means using the amplitude encoding method. 'stereo' means 
        using the inverse of the standard stereographic projection encoding method. Default set 
        to 'amplit'.
    n_copies : int, default = 1
        The number of copies to take for each quantum density. This is equivalent to taking 
        the n-fold Kronecker tensor product for each quantum density.
    measure : str, default = 'pgm'
        The measurement used to distinguish between quantum states. Possible values: 'pgm', 
        'hels'. The value 'pgm' stands for "Pretty Good Measurement", 'hels' stands for 
        "Helstrom measurement" (applicable only for binary classification). Default set to 
        'pgm'. 
    class_wgt : str, default = None        
        Applicable only when "Helstrom measurement" is selected. This is the class weights 
        assigned to the Quantum Helstrom observable terms. Possible values: 'equi', 'weighted', 
        None. 'equi' means assigning equal weights of 1/2 (equiprobable) to the two terms in 
        the Quantum Helstrom observable. 'weighted' means assigning weights equal to the 
        proportion of the number of rows in each class to the two terms in the Quantum Helstrom 
        observable. When using "Pretty Good Measurement", specify class_wgt = None. Default set 
        to None.        
    n_splits : int, default = 1
        The number of subset splits performed on the input dataset row-wise and on the number 
        of eigenvalues/eigenvectors of the Quantum Helstrom observable for optimal speed 
        performance. If 1 is given, no splits are performed. For optimal speed, recommend 
        using small values as close to 1 as possible. If memory blow-out occurs, increase 
        n_splits.
    dtype : torch.float32 or torch.float64, default = torch.float64
        The float datatype used for the elements in the Pytorch tensor dataset. Datatype has to
        be of float to ensure calculations are done in float rather than integer. To achieve
        higher n_copies without memory blow-out issues, reduce float precision, which may or may   
        not affect accuracy in a significant way.
    
    Attributes
    ----------
    classes_ : ndarray, shape (n_classes,)
        Sorted binary classes. Stored in CPU.
    qcentroids_ : tensor, size (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Quantum Centroids for each class. Stored in GPU.
    pgms_ : tensor, shape (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Pretty Good Measurement. Stored in GPU.
    hels_obs_ : tensor, size ((n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Quantum Helstrom observable. Stored in GPU.
    proj_sums_ : tensor, size (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Sum of the projectors of the Quantum Helstrom observable's eigenvectors, which has
        corresponding positive and negative eigenvalues respectively. Stored in GPU.
    hels_bound_ : float
        Helstrom bound is the upper bound of the probability that one can correctly 
        discriminate whether a quantum density is of which of the two binary quantum density 
        pattern. Stored in CPU.         
    """   
    # Initialize model hyperparameters
    def __init__(self, 
                 rescale = 1,
                 encoding = 'amplit',
                 n_copies = 1,  
                 measure = 'pgm',
                 class_wgt = None, 
                 n_splits = 1,
                 dtype = torch.float64):
        self.rescale = rescale
        self.encoding = encoding
        self.n_copies = n_copies
        self.measure = measure
        self.class_wgt = class_wgt
        self.n_splits = n_splits
        self.dtype = dtype
        
        # Raise error if dtype is not torch.float32 or torch.float64
        if self.dtype not in [torch.float32, torch.float64]:
            raise ValueError('dtype should be torch.float32 or torch.float64 only')
        
    
    # Function for kronecker tensor product for PyTorch tensors, set as global function
    global kronecker
    def kronecker(A, B):
        return torch.einsum('nab,ncd->nacbd', A, B).view(A.size(0), 
                                                         A.size(1)*B.size(1), 
                                                         A.size(2)*B.size(2))
    
    
    # Function for fit
    def fit(self, X, y):
        """Perform PGM-HQC classification with the amplitude and inverse of the standard 
        stereographic projection encoding methods, with the option to rescale the dataset prior 
        to encoding.
                
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples. An array of int or float.
        y : array-like, shape (n_samples,)
            The training input binary target values. An array of str, int or float.
            
        Returns
        -------
        self : object
            Returns self.
        """
        # Check data in X and y as required by scikit-learn v0.25
        X, y = self._validate_data(X, y, reset = True)
        
        # Ensure target array y is of non-regression type  
        # Added as required by sklearn check_estimator
        check_classification_targets(y)
            
        # Store classes and encode y into class indexes
        self.classes_, y_class_index = np.unique(y, return_inverse = True)
        
        # Number of classes, set as global variable
        global num_classes
        num_classes = len(self.classes_)
        
        # Raise error when there are more than 2 classes and Helstrom measurement is specified
        if num_classes > 2 and self.measure == 'hels':
            raise ValueError('Helstrom measurement can be applied for binary classification only')
        else:                    
            # Cast array X into a floating point tensor to ensure all following calculations below  
            # are done in float rather than integer, and send tensor X from CPU to GPU
            X = torch.tensor(X, dtype = self.dtype).cuda()
        
            # Rescale X
            X = self.rescale*X
        
            # Calculate sum of squares of each row (sample) in X
            X_sq_sum = (X**2).sum(dim = 1)
        
            # Number of rows in X
            m = X.shape[0]
        
            # Number of columns in X
            n = X.shape[1]
        
            # Calculate X' using amplitude or inverse of the standard stereographic projection 
            # encoding method
            if self.encoding == 'amplit':
                X_prime = normalize(torch.cat([X, torch.ones(m, dtype = self.dtype) \
                                               .reshape(-1, 1).cuda()], dim = 1), p = 2, dim = 1)
            elif self.encoding == 'stereo':
                X_prime = (1/(X_sq_sum + 1)).reshape(-1, 1) \
                          *(torch.cat((2*X, (X_sq_sum - 1).reshape(-1, 1)), dim = 1))
            else:
                raise ValueError('encoding should be "amplit" or "stereo"')
        
            # Number of columns in X', set as global variable
            global n_prime
            n_prime = n + 1
        
            # Function to calculate number of rows (samples) and Quantum Centroids for each class 
            def qcentroids_terms_func(i):
                # Cast array y_class_index into a tensor and send from CPU to GPU
                # Determine rows (samples) in X' belonging to either class
                X_prime_class = X_prime[torch.CharTensor(y_class_index).cuda() == i]
                                    
                # Split X' belonging to either class into n_splits subsets, row-wise
                # Send tensors from GPU to CPU and cast tensors into arrays, use np.array_split()
                # because the equivalent torch.chunk() doesn't behave similarly to np.array_split()
                X_prime_class_split_arr = np.array_split(X_prime_class.cpu().numpy(),
                                                         indices_or_sections = self.n_splits,
                                                         axis = 0)
            
                # Cast arrays back to tensors and send back from CPU to GPU
                X_prime_class_split = [torch.tensor(a, dtype = self.dtype).cuda() 
                                       for a in X_prime_class_split_arr]
            
                # Function to calculate sum of quantum densities belonging to each class, 
                # per subset split
                def X_prime_class_split_func(j):
                    # Counter for j-th split of X'
                    X_prime_class_split_jth = X_prime_class_split[j]
                
                    # Number of rows (samples) in j-th split of X'
                    m_class_split = X_prime_class_split_jth.shape[0]
                
                    # Encode vectors into quantum densities
                    density_chunk = torch.matmul(X_prime_class_split_jth.view(m_class_split, 
                                                                              n_prime, 1),
                                                 X_prime_class_split_jth.view(m_class_split, 
                                                                              1, n_prime))
                
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:
                        density_chunk = density_chunk
                    else:
                        density_chunk_copy = density_chunk
                        for b in range(self.n_copies - 1):
                            density_chunk = kronecker(density_chunk, density_chunk_copy)
                    
                    # Calculate sum of quantum densities
                    density_chunk_sum = density_chunk.sum(dim = 0)
                    return density_chunk_sum

                # Number of rows/columns in density matrix, set as global variable
                global density_nrow_ncol
                density_nrow_ncol = n_prime**self.n_copies
            
                # Initialize tensor density_class_sum
                density_class_sum = torch.zeros([density_nrow_ncol, density_nrow_ncol], 
                                                dtype = self.dtype).cuda()
                for j in range(self.n_splits):
                    # Calculate sum of quantum densities belonging to each class
                    density_class_sum = density_class_sum + X_prime_class_split_func(j)
            
                # Number of rows (samples) in X' belonging to each class
                m_class = X_prime_class.shape[0]
            
                # Function to calculate Quantum Centroid belonging to each class
                def qcentroid_func():
                    # Calculate Quantum Centroid belonging to each class
                    # Added ZeroDivisionError as required by sklearn check_estimator
                    try:
                        qcentroid = (1/m_class)*density_class_sum
                    except ZeroDivisionError:
                        qcentroid = 0 
                    return qcentroid
            
                # Calculate Quantum Centroid belonging to each class
                qcentroid_class = qcentroid_func()
                return m_class, qcentroid_class
            
            # Calculate number of rows (samples) and Quantum Centroids for each class 
            qcentroids_terms = [qcentroids_terms_func(i) for i in range(num_classes)]
            
            # Determine Quantum Centroids
            self.qcentroids_ = torch.stack([qcentroids_terms[z][1] for z in range(num_classes)], dim = 0)
            
            # When Pretty Good Measurement is specified
            if self.measure == 'pgm':
                if self.class_wgt == None:
                    # Calculate R
                    R = self.qcentroids_.sum(dim = 0)
                    
                    # Calculate square root of pseudo inverse of R
                    # Change datatype of R to float64 as the square root of a matrix calculation is highly 
                    # senstive to numerical precision/rounding
                    # Calculate pseudo inverse of R, send tensor from GPU to CPU and cast into an array
                    # Use scipy.linalg.sqrtm() to calculate square root of the pseudo inverse of R because 
                    # there is no equivalent function in PyTorch which behaves numerically similarly 
                    # Remove complex part of the matrix created due to numerical precision/rounding issues
                    # in machine language
                    # Cast array back into a tensor and send back from CPU to GPU
                    sqrt_pinv_R = torch.tensor(np.real(linalg.sqrtm(torch.pinverse(torch.as_tensor(R, dtype = \
                                              torch.float64)).cpu().numpy())), dtype = self.dtype).cuda()
                    
                    # Calculate kernel of R
                    # Change datatype of R to float64 as the kernel of a matrix calculation is highly
                    # senstive to numerical precision/rounding
                    # Send tensor from GPU to CPU and cast into an array, use scipy.linalg.null_space()
                    # to calculate kernel because there is no equivalent function in PyTorch which
                    # behaves numerically similarly
                    # Cast array back into a tensor and send back from CPU to GPU
                    ker_R = torch.tensor(linalg.null_space(torch.as_tensor(R, dtype = torch.float64).cpu() \
                                         .numpy()), dtype = self.dtype).cuda()
                    
                    # Calculate projector of kernel of R
                    proj_ker_R = torch.matmul(ker_R, ker_R.T)
                    
                    # Function to calculate Pretty Good Measurement
                    def pgm_func(a):
                        return torch.matmul(torch.matmul(sqrt_pinv_R, self.qcentroids_[a]), sqrt_pinv_R) \
                               + (1/num_classes)*proj_ker_R
                                               
                    # Calculate Pretty Good Measurement
                    self.pgms_ = torch.stack([pgm_func(a) for a in range(num_classes)], dim = 0)
                else:
                    raise ValueError('when using "pgm" measure, class_wgt should be None')
            # When Helstrom measurement is specified
            elif self.measure == 'hels':
                # Calculate quantum Helstrom observable
                if self.class_wgt == 'equi':
                    self.hels_obs_ = 0.5*(self.qcentroids_[0] - self.qcentroids_[1])
                elif self.class_wgt == 'weighted':
                    self.hels_obs_ = (qcentroids_terms[0][0]/m)*self.qcentroids_[0] \
                                     - (qcentroids_terms[1][0]/m)*self.qcentroids_[1]
                else:
                    raise ValueError('when using "hels" measure, class_wgt should be "equi" or "weighted"')
                
                # Number of rows/columns in density matrix, set as global variable
                global density_nrow_ncol
                density_nrow_ncol = self.hels_obs_.shape[0]
                
                # Calculate eigenvalues w and unit eigenvectors v of the quantum Helstrom observable
                w, v = torch.symeig(self.hels_obs_, eigenvectors = True)
                
                # Length of w
                len_w = len(w)
                
                # Initialize tensor eigval_class
                eigval_class = torch.empty_like(w, dtype = self.dtype).cuda()
                for d in range(len_w):
                    # Create a tensor of 0s and 1s to indicate positive and negative eigenvalues
                    # respectively
                    if w[d] > 0:
                        eigval_class[d] = 0
                    else:
                        eigval_class[d] = 1
                        
                # Transpose matrix v containing eigenvectors to row-wise
                eigvec = v.T
                
                # Function to calculate sum of the projectors corresponding to positive and negative
                # eigenvalues respectively
                def sum_proj_func(e):
                    # Split eigenvectors belonging to positive or negative eigenvalues into n_splits subsets
                    # Send tensors from GPU to CPU and cast tensors into arrays, use np.array_split()
                    # because the equivalent torch.chunk() doesn't behave similarly to np.array_split()
                    eigvec_class_split_arr_full = np.array_split(eigvec.cpu().numpy()[eigval_class.cpu() == e],
                                                                 indices_or_sections = self.n_splits,
                                                                 axis = 0)
                    
                    # Remove empty rows in eigvec_class_split_arr_full
                    eigvec_class_split_arr = [f for f in eigvec_class_split_arr_full if f.shape[0] > 0]
                    
                    # Cast arrays back to tensors and send back from CPU to GPU
                    eigvec_class_split = [torch.tensor(g, dtype = self.dtype).cuda()
                                          for g in eigvec_class_split_arr]
                    
                    # Function to calculate sum of the projectors corresponding to positive and negative
                    # eigenvalues respectively, per subset split
                    def eigvec_class_split_func(h):
                        # Counter for h-th split of eigvec
                        eigvec_class_split_hth = eigvec_class_split[h]
                        
                        # Number of rows (samples) in h-th split of eigvec
                        m_eigvec_class_split = eigvec_class_split_hth.shape[0]
                        
                        # Calculate projectors corresponding to positive and negative eigenvalues
                        # respectively, per subset split
                        proj_split = torch.matmul(eigvec_class_split_hth.view(m_eigvec_class_split,
                                                                              density_nrow_ncol, 1),
                                                  eigvec_class_split_hth.view(m_eigvec_class_split,
                                                                              1, density_nrow_ncol))
                        
                        # Calculate sum of projectors
                        proj_split_sum = proj_split.sum(dim = 0)
                        return proj_split_sum
                    
                    # Determine length of eigvec_class_split_arr
                    eigvec_class_split_arr_len = len(eigvec_class_split_arr)
                    
                    # Initialize tensor proj_class_sum
                    proj_class_sum = torch.zeros([density_nrow_ncol, density_nrow_ncol],
                                                 dtype = self.dtype).cuda()
                    for h in range(eigvec_class_split_arr_len):
                        # Calculate sum of the projectors corresponding to positive and negative eigenvalues
                        # respectively
                        proj_class_sum = proj_class_sum + eigvec_class_split_func(h)
                    return proj_class_sum
                
                # Calculate sum of the projectors corresponding to positive and negative eigenvalues
                # respectively
                self.proj_sums_ = torch.stack([sum_proj_func(0), sum_proj_func(1)], dim = 0)
                
                # Calculate Helstrom bound
                self.hels_bound_ = (qcentroids_terms[0][0]/m)*torch.einsum('ij,ji->', self.qcentroids_[0],
                                                                           self.proj_sums_[0]).item() \
                                   + (qcentroids_terms[1][0]/m)*torch.einsum('ij,ji->', self.qcentroids_[1],
                                                                             self.proj_sums_[1]).item()
            # When Pretty Good Measurement or Helstrom measurement is misspecified
            else:
                raise ValueError('measure should be "pgm" or "hels"')
        return self

           
    # Function for predict_proba
    def predict_proba(self, X):
        """Performs PMG-HQC classification on X and returns the trace of the dot product of the 
        densities and the POV (positive operator-valued) measure, i.e. the class probabilities.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.       
            
        Returns
        -------
        trace_matrix : tensor, size (n_samples, n_classes)
            Each column corresponds to the trace of the dot product of the densities and the POV 
            (positive operator-valued) measure for each class, i.e. each column corresponds to the 
            class probabilities. A tensor of float. Stored in GPU.
        """
        # Send tensors self.pgms_ and self.proj_sums_ from GPU to CPU and cast into an array, and
        # check if fit had been called
        if self.measure == 'pgm':
            self.pgms_arr_ = self.pgms_.cpu().numpy()
            check_is_fitted(self, ['pgms_arr_'])
        else:
            self.proj_sums_arr_ = self.proj_sums_.cpu().numpy()
            check_is_fitted(self, ['proj_sums_arr_'])
               
        # Check data in X as required by scikit-learn v0.25
        X = self._validate_data(X, reset = False)
                 
        # Cast array X into a floating point tensor to ensure all following calculations below  
        # are done in float rather than integer, and send tensor X from CPU to GPU
        X = torch.tensor(X, dtype = self.dtype).cuda()
        
        # Rescale X
        X = self.rescale*X        
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(dim = 1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]

        # Calculate X' using amplitude or inverse of the standard stereographic projection 
        # encoding method
        if self.encoding == 'amplit':
            X_prime = normalize(torch.cat([X, torch.ones(m, dtype = self.dtype) \
                                           .reshape(-1, 1).cuda()], dim = 1), p = 2, dim = 1)
        elif self.encoding == 'stereo':
            X_prime = (1/(X_sq_sum + 1)).reshape(-1, 1) \
                      *(torch.cat((2*X, (X_sq_sum - 1).reshape(-1, 1)), dim = 1))
        else:
            raise ValueError('encoding should be "amplit" or "stereo"')
                       
        # Function to calculate trace values for each class
        def trace_func(i):
            # Split X' into n_splits subsets, row-wise
            # Send tensors from GPU to CPU and cast tensors into arrays, use np.array_split()
            # because the equivalent torch.chunk() doesn't behave similarly to np.array_split()
            X_prime_split_arr_full = np.array_split(X_prime.cpu().numpy(),
                                                    indices_or_sections = self.n_splits,
                                                    axis = 0)
            
            # Remove empty rows in X_prime_split_arr_full
            X_prime_split_arr = [a for a in X_prime_split_arr_full if a.shape[0] > 0]

            # Cast arrays back to tensors and send back from CPU to GPU
            X_prime_split = [torch.tensor(q, dtype = self.dtype).cuda() for q in X_prime_split_arr]
            
            # Function to calculate trace values for each class, per subset split
            def trace_split_func(j):
                # Counter for j-th split X'
                X_prime_split_jth = X_prime_split[j]
                
                # Number of rows (samples) in j-th split X'
                X_prime_split_m = X_prime_split_jth.shape[0]
                
                # Encode vectors into quantum densities
                density_chunk = torch.matmul(X_prime_split_jth.view(X_prime_split_m, n_prime, 1),
                                             X_prime_split_jth.view(X_prime_split_m, 1, n_prime))
                
                # Calculate n-fold Kronecker tensor product
                if self.n_copies == 1:
                    density_chunk = density_chunk
                else:
                    density_chunk_copy = density_chunk
                    for b in range(self.n_copies - 1):
                        density_chunk = kronecker(density_chunk, density_chunk_copy)
                        
                # When Pretty Good Measurement is specified
                if self.measure == 'pgm':
                    # Calculate trace of the dot product of density of each row and Pretty Good
                    # Measurement
                    trace_class_split = torch.einsum('bij,ji->b', density_chunk, self.pgms_[i])
                # When Helstrom measurement is specified
                else:
                    # Calculate trace of the dot product of density of each row and sum of 
                    # projectors with corresponding positive and negative eigenvalues respectively
                    trace_class_split = torch.einsum('bij,ji->b', density_chunk, self.proj_sums_[i])
                return trace_class_split
            
            # Determine length of X_prime_split_arr
            X_prime_split_arr_len = len(X_prime_split_arr)

            # Initialize tensor trace_class
            trace_class = torch.empty([0], dtype = self.dtype).cuda()
            for j in range(X_prime_split_arr_len):
                # Calculate trace values for each class, per subset split
                trace_class = torch.cat([trace_class, trace_split_func(j)], dim = 0)
            return trace_class
        
        # Calculate trace values for each class
        trace_matrix = torch.stack([trace_func(i) for i in range(num_classes)], dim = 1)
        return trace_matrix
                
    
    # Function for predict
    def predict(self, X):
        """Performs PGM-HQC classification on X and returns the classes.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.
            
        Returns
        -------
        self.classes_[predict_trace_index] : array-like, shape (n_samples,)
            The predicted binary classes. An array of str, int or float.
        """
        # Determine column index with the higher trace value in trace_matrix
        # If both columns have the same trace value, returns column index 1, which is different 
        # to np.argmax() which returns column index 0
        predict_trace_index = torch.argmax(self.predict_proba(X), axis = 1)
        # Returns the predicted binary classes
        return self.classes_[predict_trace_index.cpu().numpy()]

In [2]:
# appendicitis dataset (7 features, 106 rows)
import pandas as pd

df = pd.read_csv('appendicitis.tsv',delimiter='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

In [3]:
# Check F1 score and Helstrom bound values for various rescale and n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=3, encoding='stereo', measure='hels', class_wgt='weighted', n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted'), model.hels_bound_

(0.7520661157024794, 0.8772541781266531)

In [4]:
# banana dataset (2 features, 5300 rows)
import pandas as pd

df = pd.read_csv('banana.tsv', sep='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

In [5]:
# Check F1 score and Helstrom bound values for various rescale and n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=4, encoding='stereo', measure='hels', class_wgt='weighted', n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted'), model.hels_bound_

(0.858978398722441, 0.7732936040410455)

In [6]:
# iris dataset (5 features, 150 rows)
import pandas as pd

df = pd.read_csv('iris.tsv', sep='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

In [7]:
# Check if class imbalance
df['target'].value_counts()

2    50
1    50
0    50
Name: target, dtype: int64

In [8]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

# Check trace values sum to 1 for first 5 rows
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=1, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
torch.cat([model.predict_proba(X_test), torch.sum(model.predict_proba(X_test), dim=1).reshape(-1,1)], dim=1)[0:5,:]

tensor([[0.2714, 0.3513, 0.3773, 1.0000],
        [0.2244, 0.3658, 0.4098, 1.0000],
        [0.5722, 0.2603, 0.1675, 1.0000],
        [0.2334, 0.3523, 0.4143, 1.0000],
        [0.2178, 0.3588, 0.4234, 1.0000]], device='cuda:0')

In [9]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=1, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8940170940170941

In [10]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=2, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8940170940170941

In [11]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=3, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8940170940170941

In [12]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=4, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8940170940170941

In [13]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=0.5, n_copies=5, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8940170940170941

In [14]:
# Testing using scikit-learn's GridSearchCV
from sklearn.model_selection import GridSearchCV

# Create rescale hyperparamter list [0.1, 0.5, 1, 1.5,...,10.0]
rescale_list1 = [0.1]
rescale_list2 = np.linspace(0.5, 10, 20).tolist()
rescale_list1.extend(rescale_list2)

param_grid = {'rescale':rescale_list1, 'n_copies':[1, 2, 3, 4], 'encoding':['amplit', 'stereo'], 'measure':['pgm'], 'class_wgt':[None]}
models = GridSearchCV(PGMHQC_gpu_dtype(n_splits=1, dtype=torch.float32), param_grid, scoring='f1_weighted').fit(X_train, y_train)

In [15]:
# Best F1 score
best_model = models.best_estimator_
y_hat = best_model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

1.0

In [16]:
# Best hyperparameter combination
models.best_params_

{'class_wgt': None,
 'encoding': 'stereo',
 'measure': 'pgm',
 'n_copies': 4,
 'rescale': 0.1}

In [17]:
# balance-scale dataset (5 features, 625 rows)
import pandas as pd

df = pd.read_csv('balance-scale.tsv', sep='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

In [18]:
# Check if class imbalance
df['target'].value_counts()

2    288
1    288
0     49
Name: target, dtype: int64

In [19]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

# Check trace values sum to 1 for first 5 rows
model = PGMHQC_gpu_dtype(rescale=1, n_copies=2, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
torch.cat([model.predict_proba(X_test), torch.sum(model.predict_proba(X_test), dim=1).reshape(-1,1)], dim=1)[0:5,:]

tensor([[0.3146, 0.3284, 0.3570, 1.0000],
        [0.3154, 0.3281, 0.3566, 1.0000],
        [0.3152, 0.3429, 0.3419, 1.0000],
        [0.3229, 0.3163, 0.3607, 0.9999],
        [0.3167, 0.3418, 0.3415, 1.0000]], device='cuda:0')

In [20]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1, n_copies=1, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8367315008833559

In [21]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1, n_copies=2, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8447445962196664

In [22]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1, n_copies=3, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8605017010473518

In [30]:
### SCORE IS SLIGHTLY DIFFERENT BETWEEN FLOAT32 AND FLOAT64 (0.8676190476190477 vs. 0.8682253173918403) ###
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1, n_copies=4, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8676190476190477

In [31]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1, n_copies=5, encoding='stereo', measure='pgm', class_wgt=None, n_splits=50, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8678604178430266

In [32]:
# Testing using scikit-learn's GridSearchCV
from sklearn.model_selection import GridSearchCV

# Create rescale hyperparamter list [0.1, 0.5, 1, 1.5,...,10.0]
rescale_list1 = [0.1]
rescale_list2 = np.linspace(0.5, 10, 20).tolist()
rescale_list1.extend(rescale_list2)

param_grid = {'rescale':rescale_list1, 'n_copies':[1, 2, 3, 4], 'encoding':['amplit', 'stereo'], 'measure':['pgm'], 'class_wgt':[None]}
models = GridSearchCV(PGMHQC_gpu_dtype(n_splits=1, dtype=torch.float32), param_grid, scoring='f1_weighted').fit(X_train, y_train)

In [33]:
# Best F1 score
best_model = models.best_estimator_
y_hat = best_model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.9106013291733841

In [34]:
# Best hyperparameter combination
models.best_params_

{'class_wgt': None,
 'encoding': 'amplit',
 'measure': 'pgm',
 'n_copies': 2,
 'rescale': 2.0}

In [35]:
# new-thyroid dataset (6 features, 215 rows)
import pandas as pd

df = pd.read_csv('new-thyroid.tsv', sep='\t')
X = df.drop('target', axis=1).values
y = df['target'].values

In [36]:
# Check if class imbalance
df['target'].value_counts()

1    150
2     35
3     30
Name: target, dtype: int64

In [37]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=4)

# Check trace values sum to 1 for first 5 rows
model = PGMHQC_gpu_dtype(rescale=1.5, n_copies=3, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
torch.cat([model.predict_proba(X_test), torch.sum(model.predict_proba(X_test), dim=1).reshape(-1,1)], dim=1)[0:5,:]

tensor([[0.3336, 0.3326, 0.3336, 0.9998],
        [0.3313, 0.3300, 0.3386, 0.9998],
        [0.3337, 0.3324, 0.3337, 0.9998],
        [0.3338, 0.3325, 0.3336, 0.9998],
        [0.3338, 0.3328, 0.3331, 0.9998]], device='cuda:0')

In [38]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1.5, n_copies=1, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8549863029667192

In [39]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1.5, n_copies=2, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8549863029667192

In [47]:
### SCORE IS SLIGHTLY DIFFERENT BETWEEN FLOAT32 AND FLOAT64 (0.8314708547266686 vs. 0.8549863029667192) ###
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1.5, n_copies=3, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8314708547266686

In [48]:
# Check F1 score for various n_copies values
model = PGMHQC_gpu_dtype(rescale=1.5, n_copies=4, encoding='stereo', measure='pgm', class_wgt=None, n_splits=1, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8549863029667192

In [50]:
# Check F1 score for various n_copies values
# 150+35+30 = 215
model = PGMHQC_gpu_dtype(rescale=1.5, n_copies=5, encoding='stereo', measure='pgm', class_wgt=None, n_splits=100, dtype=torch.float32).fit(X_train, y_train)
y_hat = model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.8549863029667192

In [51]:
# Testing using scikit-learn's GridSearchCV
from sklearn.model_selection import GridSearchCV

# Create rescale hyperparamter list [0.1, 0.5, 1, 1.5,...,10.0]
rescale_list1 = [0.1]
rescale_list2 = np.linspace(0.5, 10, 20).tolist()
rescale_list1.extend(rescale_list2)

# 150+35+30 = 215
param_grid = {'rescale':rescale_list1, 'n_copies':[1, 2, 3, 4], 'encoding':['amplit', 'stereo'], 'measure':['pgm'], 'class_wgt':[None]}
models = GridSearchCV(PGMHQC_gpu_dtype(n_splits=1, dtype=torch.float32), param_grid, scoring='f1_weighted').fit(X_train, y_train)

In [52]:
# Best F1 score
best_model = models.best_estimator_
y_hat = best_model.predict(X_test)

from sklearn import metrics
metrics.f1_score(y_test, y_hat, average='weighted')

0.9516311369509043

In [53]:
# Best hyperparameter combination
models.best_params_

{'class_wgt': None,
 'encoding': 'amplit',
 'measure': 'pgm',
 'n_copies': 2,
 'rescale': 0.1}